# Models Evaluator
Evaluate all the models on all the test datasets

**Authors**

`Marco Alecci <https://github.com/MarcoAlecci>`

`Francesco Marchiori <https://github.com/FrancescoMarchiori>`

`Luca Martinelli <https://github.com/luca-martinelli-09>`

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/luca-martinelli-09/orco-gan/blob/main/modelEvaluator.ipynb)

## General Setup

In [46]:
import os

if not os.path.exists("./datasets"):
    !git clone "https://github.com/luca-martinelli-09/orco-gan.git"

    %cd orco-gan/

In [47]:
import sys
IN_COLAB = 'google.colab' in sys.modules

googleModelsDir = None
googleAdvSamplesDir = None

if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

  googleModelsDir = "/content/drive/MyDrive/Università/Magistrale/II Anno/I Semestre/Advanced Topics in Computer and Network Security/Project/Models"
  googleAdvSamplesDir = None

In [48]:
import torch
import pandas as pd
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)

PyTorch Version: 1.10.1+cu113
Torchvision Version: 0.11.2+cu113


In [49]:
# Detect if we have a GPU available
print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CUDA available: True


### Set a manual seed

In [50]:
SEED = 151836

def setSeed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

setSeed(SEED)

## Utils

In [51]:
def printGPUStats():
    print('Using device:', device)
    print()

    # Additional Info when using cuda
    if device.type == 'cuda':
        print(torch.cuda.get_device_name(0))
        print('[💻 MEMORY USAGE]')
        print('[📌 ALLOCATED]', round(
            torch.cuda.memory_allocated(0) / 1024 ** 3, 1), 'GB')
        print('[🧮 CACHED]', round(
            torch.cuda.memory_reserved(0) / 1024 ** 3, 1), 'GB')


In [52]:
def getSubDirs(dir):
    return [x for x in os.listdir(dir) if os.path.isdir(os.path.join(dir, x))]

In [53]:
def getClassPercents(sizes):
    totalSize = np.sum(np.array(sizes))
    percents = []
    for size in sizes:
        percents.append(int(round((size / totalSize) * 100)))
    
    return percents

In [54]:
def getBestScores(hist, key, min=False):
    scores = [x[key] for x in hist]

    if min:
        i = np.argmin(np.array(scores))
    else:
        i = np.argmax(np.array(scores))

    return hist[i], i

In [55]:
def getMeanAndSDT(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

## Settings

In [56]:
datasetsDir = "./datasets"
modelsDir = googleModelsDir if googleModelsDir else "./models"
adversarialsDir = googleAdvSamplesDir if googleAdvSamplesDir else "./adversarial_samples"

inputSize = 224 # Specified for alexnet, resnet, vgg

## Models evaluations

In [57]:
def getScores(labels, predicted):
    acc = torch.sum(predicted == labels) / len(predicted)

    tp = (labels * predicted).sum()
    tn = ((1 - labels) * (1 - predicted)).sum()
    fp = ((1 - labels) * predicted).sum()
    fn = (labels * (1 - predicted)).sum()

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2 * (precision * recall) / (precision + recall)

    return acc, precision, recall, f1

In [58]:
def evaluateModel(model, dataloader, attack=None):
    model.eval()
    labelsOutputs = torch.tensor([]).to(device, non_blocking=True)
    labelsTargets = torch.tensor([]).to(device, non_blocking=True)

    setSeed(SEED)
    for inputs, labels in dataloader:
        inputs = attack(inputs, labels) if attack else inputs
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        labelsOutputs = torch.cat([labelsOutputs, preds], dim=0)
        labelsTargets = torch.cat([labelsTargets, labels], dim=0)
    
    acc, precision, recall, f1 = getScores(labelsTargets, labelsOutputs)

    return {
        "acc": acc.cpu().numpy(),
        "precision": precision.cpu().numpy(),
        "recall": recall.cpu().numpy(),
        "f1": f1.cpu().numpy()
    }


### Informations about models

In [59]:
print("[🧠 MODELS INFORMATION]")

modelsInformation = []

for dataset in getSubDirs(modelsDir):
    print("\n" + "-" * 15)
    print("[🗃️ DATASET] {}".format(dataset))

    datasetDir = os.path.join(modelsDir, dataset)

    for modelType in getSubDirs(datasetDir):
        print("\n[🧮 MODEL TYPE] {}".format(modelType))

        modelsTypeDir = os.path.join(datasetDir, modelType)
        
        for model in os.listdir(modelsTypeDir):
            print("\n\t[🧠 MODEL] {}".format(model))

            path = os.path.join(modelsTypeDir, model)

            checkpoint = torch.load(path)
            
            bestScore, i = getBestScores(checkpoint["scores_history"], "f1")
            classBalancing = getClassPercents(checkpoint["dataset_sizes"])
            balancingStr = "/".join([str(x) for x in classBalancing])

            modelsInformation.append({
                "dataset": dataset,
                "model": checkpoint["model_name"],
                "epochs": len(checkpoint["scores_history"]),
                "balancing": balancingStr,
                "f-score": bestScore["f1"],
            })

            print("\tModel:", checkpoint["model_name"])
            print("\tEpochs:", len(checkpoint["scores_history"]))
            print("\tBalancing:", classBalancing)
            print("\tBest epoch:", i)
            print("\tBest F-Score:", bestScore["f1"])
            print("\tHistory:", [float(x["f1"]) for x in checkpoint["scores_history"]])

            torch.cuda.empty_cache()

modelsInformationDF = pd.DataFrame(modelsInformation)

printGPUStats()

[🧠 MODELS INFORMATION]

---------------
[🗃️ DATASET] bing

[🧮 MODEL TYPE] alexnet

	[🧠 MODEL] alexnet_20_80_all.pt
	Model: alexnet
	Epochs: 38
	Balancing: [20, 80]
	Best epoch: 17
	Best F-Score: 0.9436486
	History: [0.9077625274658203, 0.923933207988739, 0.929311990737915, 0.9321871995925903, 0.9340866208076477, 0.9332079291343689, 0.9357277750968933, 0.9391635060310364, 0.9417383074760437, 0.9417383074760437, 0.942638635635376, 0.942638635635376, 0.942638635635376, 0.942638635635376, 0.942638635635376, 0.942638635635376, 0.942638635635376, 0.9436485767364502, 0.9436485767364502, 0.9427481293678284, 0.9427481293678284, 0.9428571462631226, 0.9419600367546082, 0.9419600367546082, 0.9419600367546082, 0.9419600367546082, 0.9419600367546082, 0.9401708841323853, 0.9392789602279663, 0.9383886456489563, 0.9383886456489563, 0.9383886456489563, 0.9383886456489563, 0.9383886456489563, 0.9383886456489563, 0.9383886456489563, 0.9383886456489563, 0.9383886456489563]

	[🧠 MODEL] alexnet_30_70_all.pt


In [60]:
modelsInformationDF

,dataset,model,epochs,balancing,f-score
0,bing,alexnet,38,20/80,0.9436486
1,bing,alexnet,24,30/70,0.9522863
2,bing,alexnet,23,40/60,0.94939274
3,bing,alexnet,42,50/50,0.95114654
4,bing,resnet,49,20/80,0.9659201
5,bing,resnet,25,30/70,0.98513377
6,bing,resnet,33,40/60,0.98701304
7,bing,resnet,26,50/50,0.98217815
8,bing,vgg,28,20/80,0.9765166
9,bing,vgg,38,30/70,0.98128074


### Evaluations

In [61]:
def evaluateModelsOnDataset(datasetFolder, datasetInfo):
    global modelsDir

    modelsEvals = []

    # Get the images and calculate mean and standard deviation
    toTensor = transforms.Compose([transforms.ToTensor()])
    testDataset = ImageLimitedDataset(
        datasetFolder, transform=toTensor, use_cache=False, check_images=False)
    
    for cls in testDataset.classes:
        cls_index = testDataset.class_to_idx[cls]
        num_cls = np.count_nonzero(
            np.array(testDataset.targets) == cls_index)
        
        print("\t[🧮 # ELEMENTS] {}: {}".format(cls, num_cls))
    
    setSeed(SEED)
    testDataLoader = DataLoader(
        testDataset, batch_size=16, shuffle=True, num_workers=0)
    
    mean, std = getMeanAndSDT(testDataLoader)

    # Setup for normalization
    dataTransform = transforms.Compose([
        transforms.Resize(inputSize),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    torch.cuda.empty_cache()

    testDataset = ImageLimitedDataset(
        datasetFolder, transform=dataTransform, use_cache=True, check_images=False)

    setSeed(SEED)
    testDataLoader = DataLoader(
        testDataset, batch_size=16, shuffle=True, num_workers=0)
    
    # Evaluate every model
    for root, _, fnames in sorted(os.walk(modelsDir, followlinks=True)):
        for fname in sorted(fnames):
            path = os.path.join(root, fname)

            try:
                modelData = torch.load(path)
            except:
                continue

            modelDataset = modelData["dataset"]
            modelName = modelData["model_name"]
            modelPercents = "/".join([str(x)
                                     for x in getClassPercents(modelData["dataset_sizes"])])

            print()
            print("[🧮 EVALUATING] {} - {} {}".format(
                modelDataset,
                modelName,
                modelPercents
            ))

            modelToTest = modelData["model"].to(device, non_blocking=True)

            setSeed(SEED)
            scores = evaluateModel(modelToTest, testDataLoader)

            modelsEvals.append({
                    "dataset": datasetInfo["dataset"],
                    "isMath": datasetInfo["math"],
                    "attack": datasetInfo["attack"],
                    "advModel": datasetInfo["model"],
                    "advBalancing": datasetInfo["balancing"],

                    "model": modelName,
                    "modelDataset": modelDataset,
                    "balancing": modelPercents,
                    "acc": scores["acc"],
                    "precision": scores["precision"],
                    "recall": scores["recall"],
                    "f1": scores["f1"],
                })
            
            print("\tAcc: {:.4f}".format(scores["acc"]))
            print("\tPre: {:.4f}".format(scores["precision"]))
            print("\tRec: {:.4f}".format(scores["recall"]))
            print("\tF-Score: {:.4f}".format(scores["f1"]))

            torch.cuda.empty_cache()
        
    return modelsEvals


In [62]:
from imageLimitedDataset import ImageLimitedDataset

print("[🧠 MODELS EVALUATION - NO ATTACKS]")

modelsEvals = []

# Evaluate models on test folders
for dataset in getSubDirs(datasetsDir):
    print("\n" + "-" * 15)
    print("[🗃️ TEST DATASET] {}".format(dataset))
    
    datasetDir = os.path.join(datasetsDir, dataset)
    testDir = os.path.join(datasetDir, "test")

    advDatasetInfo = {
        "dataset": dataset,
        "math": None,
        "attack": None,
        "balancing": None,
        "model": None,
    }

    evals = evaluateModelsOnDataset(testDir, advDatasetInfo)
    modelsEvals.extend(evals)

[🧠 MODELS EVALUATION - NO ATTACKS]

---------------
[🗃️ TEST DATASET] bing
	[🧮 # ELEMENTS] cat: 1000
	[🧮 # ELEMENTS] dog: 1000

[🧮 EVALUATING] bing - alexnet 20/80


TypeError: string indices must be integers

In [ ]:
print("[🧠 MODELS EVALUATION - NON MATH ATTACKS]")

# Evaluate models on non math attacks folders
for dataset in getSubDirs(adversarialsDir):
    datasetDir = os.path.join(adversarialsDir, dataset)
    nonMathAdvDir = os.path.join(datasetDir, "nonMath")

    for attack in getSubDirs(nonMathAdvDir):
        print("\n" + "-" * 15)
        print("[🗃️ ADVERSARIAL DATASET] {}/{}".format(dataset, attack))

        attackDir = os.path.join(nonMathAdvDir, attack)

        advDatasetInfo = {
            "dataset": dataset,
            "math": False,
            "attack": attack,
            "balancing": None,
            "model": None,
        }

        evals = evaluateModelsOnDataset(attackDir, advDatasetInfo)
        modelsEvals.extend(evals)


In [ ]:
print("[🧠 MODELS EVALUATION - MATH ATTACKS]")

# Evaluate models on math attacks folders
for dataset in getSubDirs(adversarialsDir):
    datasetDir = os.path.join(adversarialsDir, dataset)
    nonMathAdvDir = os.path.join(datasetDir, "math")

    if not os.path.exists(nonMathAdvDir):
        continue

    for attack in getSubDirs(nonMathAdvDir):
        attackDir = os.path.join(nonMathAdvDir, attack)

        for advModel in getSubDirs(attackDir):
            advModelDir = os.path.join(attackDir, advModel)

            for advBalancing in getSubDirs(advModelDir):
                advDatasetDir = os.path.join(advModelDir, advBalancing)

                print("\n" + "-" * 15)
                print("[🗃️ ADVERSARIAL DATASET] {}/{}/{}/{}".format(dataset, attack, advModel, advBalancing))

                advDatasetInfo = {
                    "dataset": dataset,
                    "math": True,
                    "attack": attack,
                    "balancing": advBalancing.replace("_", "/"),
                    "model": advModel,
                }

                evals = evaluateModelsOnDataset(advDatasetDir, advDatasetInfo)
                modelsEvals.extend(evals)

In [ ]:
modelsEvalsDF = pd.DataFrame(modelsEvals)

In [ ]:
modelsEvalsDF

### Save evaluations

In [ ]:
modelsEvalsDF.to_csv("modelsEvaluations.csv")